# Thinking1什么是近似最近邻查找，常用的方法有哪些？
    
       一、近似最近邻(ANN),相似于最近邻的一种方式，以最近邻的思想解决海量数据下运算时间过长问题(类似降维？)，但是返回的不是最相似的前N个向量，而是相似的向量。
       二、常用方法：LSH，MinHash，MinHashLSH，LSHForest，SimHash
        

# Thinking2 为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度？
       Jaccard相似度：交集/并集；MinHash值：  特征矩阵按行进行随机排列后，第一个列值为1(不为空)的行的行号；
       
       计算Pa和Pb的MinHash时一共有三种可能：1.两列都是1；2.两列都是0；3.一列是1一列是0。当a、b为0时不符合MinHash的计算标准，所以只剩下1,2两种可能，P = a行的个数/所有行的个数。

# Thinking3 SimHash在计算文档相似度的作用是怎样的？
    一.计算过程：
       1.初始化SimHash并设置位数；
       2.分词抽取n-gram，计算特征向量并赋予权重；
       3.加权计算特征hashcode编码；
       4.将各个特征向量的加权结果累加，变成一个序列串；
       5.计算最后得到的SimHash，如果该位大于0，则设为1；否则设为0；

    二、如何通过文档的SimHash计算文档之间的相似度：通过海明距离(两个SimHash不相同的个数)计算，距离越小相似度越高。

# Thinking4 为什么YouTube采用期望观看时间作为评估指标？
    YouTobe是视频网站，CTR指标对于YouTube来说无法带来具体收益，用户虽然点开视频但不一定会看完视频，具有一定的欺骗性；
    而期望观看时间（odds）则能更好的预测收益，时间越长视频可投放的广告数量就越多广告时间也就约长，YouTube的收益就越大。

# Action：  使用MinHashLSHForest对微博新闻句子进行检索 weibos.txt针对某句话进行Query，查找Top-3相似的句子

In [38]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import re

In [148]:
# 读取文本
weibo = open('./L6/weibos.txt', 'r', encoding='utf8').read()
# print(weibo)
# print('='*50)
# 以，。！？#作为分隔，去掉\n换行符号
sentences = re.split('[。！？#]', weibo.replace('\n', ''))
sentences

['',
 '斯科拉里愿意执教国足',
 '上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用',
 ' \u200b国足输给叙利亚之后，里皮辞职',
 '谁将成为新主帅，成为广大球迷关注的焦点',
 '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里',
 ' \u200b据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决',
 '据了解',
 '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战',
 '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利',
 '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话',
 '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商',
 '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……',
 '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足',
 '老头有点意思，凡是里皮干过的地方，他就想试试',
 '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛',
 '可以试试',
 '斯科拉里的水平，还不如里皮',
 '斯科拉里，看好的不是国足，而是年薪…… \u200b非常应该辞职',
 '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有',
 '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了',
 '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光',
 '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队',
 '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去',
 '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛',
 '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州',
 '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，

In [149]:
# 设置停用词
stopwords = [line.strip() for line in open('./stopwords.txt','r',encoding='utf-8').readlines()]
stopwords

[',',
 '?',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎哟',
 '唉',
 '俺',
 '俺们',
 '按',
 '按照',
 '吧',
 '吧哒',
 '把',
 '罢了',
 '被',
 '本',
 '本着',
 '比',
 '比方',
 '比如',
 '鄙人',
 '彼',
 '彼此',
 '边',
 '别',
 '别的',
 '别说',
 '并',
 '并且',
 '不比',
 '不成',
 '不单',
 '不但',
 '不独',
 '不管',
 '不光',
 '不过',
 '不仅',
 '不拘',
 '不论',
 '不怕',
 '不然',
 '不如',
 '不特',
 '不惟',
 '不问',
 '不只',
 '朝',
 '朝着',
 '趁',
 '趁着',
 '乘',
 '冲',
 '除',
 '除此之外',
 '除非',
 '除了',
 '此',
 '此间',
 '此外',
 '从',
 '从而',
 '打',
 '待',
 '但',
 '但是',
 '当',
 '当着',
 '到',
 '得',
 '的',
 '的话',
 '等',
 '等等',
 '地',
 '第',
 '叮咚',
 '对',
 '对于',
 '多',
 '多少',
 '而',
 '而况',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '尔后',
 '反过来',
 '反过来说',
 '反之',
 '非但',
 '非徒',
 '否则',
 '嘎',
 '嘎登',
 '该',
 '赶',
 '个',
 '各',
 '各个',
 '各位',
 '各种',
 '各自',
 '给',
 '根据',
 '跟',
 '故',
 '故此',
 '固然',
 '关于',
 '管',
 '归',
 '果然',
 '果真',
 '过',
 '哈',
 '哈哈',
 '呵',
 '和',
 '何',
 '何处',
 '何况',
 '何时',
 '嘿',
 '哼',
 '哼唷',
 '呼哧',
 '乎',
 '哗',
 '还是',
 '还有',
 '换句话说',
 '换言之',
 '或',
 

In [150]:
# 分词函数
def get_item_str(text):
    item_str = "" 
    items=(jieba.cut(text)) 
    for item in list(items):
        #去除停用词
        if item not in list(stopwords):  
            item_str += item
            item_str += " "
#             vectorizer = TfidfVectorizer()
#             vectorizers = vectorizer.fit_transform(item_str).toarray()

    return item_str

In [152]:
# 进行分词
documents = []
for text in sentences:
    item_str = get_item_str(text)
#     print(item_str)
    documents.append(item_str)
documents

['',
 '斯科拉里 愿意 执教 国 足 ',
 '一届 里 皮 芾 尾 很大 机会 入 世界杯 一届 几个 能用 大力 归化 没用 ',
 '  \u200b 国 足 输给 叙利亚 之后 里 皮 辞职 ',
 '成为 新 主帅 成为 广大 球迷 关注 焦点 ',
 '目前 舆论 方面 倾向 三个 人 山东鲁能 主帅 李霄鹏 武汉 卓尔 主帅 李铁 前 广州 恒大 主帅 斯科拉里 ',
 '  \u200b 了解 中国足协 态度 里 皮 宣布 请辞 已经 去意 已决 ',
 '了解 ',
 '比赛 当晚 太太 西蒙内 塔 女士 儿子 小里 皮 现场 台上 观战 ',
 '辞职 后 里 皮 改变 原有 计划 赛后 第二天 他会 迪拜 直接 飞 回 意大利 ',
 '意味着 本来 打算 球队 管理层 中国足协 高层 赛后 第一 时间 进行 辞职 对话 ',
 '辞职 以后 善后工作 包括 合同 问题 沟通 工作 日后 双方 进一步 协商 ',
 '回顾 一下 国 足 历届 外籍 教练 里 皮 佩兰 卡马乔 杜伊 科维奇 阿里 汉 米 卢 ',
 '之前 一个 一个 有名 之后 一个 一个 水 国足 踢 不好 完全 足协 问题 足协 解散 重组 天王老子 请来 不行 斯科拉里 想 执教 中国 国 足 ',
 '老头 有点 意思 凡是 里 皮 干过 地方 想 试试 ',
 '老头 世界杯 冠军 教头 折 中国 丢人 里 皮 折 ',
 '试试 ',
 '斯科拉里 水平 里 皮 ',
 '斯科拉里 看好 国 足 年薪   \u200b 应该 辞职 ',
 '中国 足球 需要 名帅 需要 外籍 教练 一点儿 毛用 ',
 '施拉普纳 现在 二十余年 间 中国 足球 大踏步 倒退 一点儿 杀 不住 车 奶奶 刹车 系统 坏 ',
 '穿着 几百块 钱 球衣 几千块 钱 球鞋 几万块 钱 包 几十万 包机 几百万 年薪 赛后 叙利亚 主教练 更衣室 里 每个 队员 一个 耳光 ',
 '主教练 说 赛前 老子 再三 交代 一场 无论如何 赢 中国队 ',
 '中国 援助 粮食 美金 援助 国家 狗日 些 吃 土去 ',
 '球员 委屈 说 七十多 分钟 晓得 龟儿子 家 球门 踢 ',
 '里 皮 辞职 返回 意大利 助教 马达 洛尼 随队 返回 广州 ',
 '马达 洛尼 接受

In [153]:
# 创建MinHash计算签名矩阵（signature），近似估计原矩阵的Jaccard相似度
def get_minhash(text):
    temp = MinHash()
    for t in text:
        temp.update(t.encode('utf-8'))
    return temp

In [154]:
# 创建LSHForest
minhash = []
lshforest = MinHashLSHForest()
for i in range(len(documents)):
    # 生成minhash
    temp = get_minhash(documents[i])
    minhash.append(temp)
    
    # 训练lshforest
    lshforest.add(i, temp)
# 创建index
lshforest.index()


In [155]:
query = '这一届，没几个能用的，除非大力归化，谁来都没用。'
# 对query进行分词
query_words = get_item_str(query)
# 计算query的MinHash
minhash_query = get_minhash(query_words)

# 查询lshforest中与m1相似的Top-K个邻居
result = lshforest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3邻居:", result)

2 0.578125 一届里皮芾尾很大机会入世界杯一届几个能用大力归化没用
4 0.1328125 成为新主帅成为广大球迷关注焦点
5 0.1015625 目前舆论方面倾向三个人山东鲁能主帅李霄鹏武汉卓尔主帅李铁前广州恒大主帅斯科拉里
Top 3邻居: [2, 4, 5]


In [156]:
query2 = '斯科拉里的水平，还不如里皮。斯科拉里，看好的不是国足，而是年薪……'
# 对query进行分词
query_words2 = get_item_str(query2)
# 计算query的MinHash
minhash_query2 = get_minhash(query_words2)

# 查询lshforest中与m1相似的Top-K个邻居
result2 = lshforest.query(minhash_query2, 3)
for i in range(len(result2)):
    print(result2[i], minhash_query2.jaccard(minhash[result2[i]]), documents[result2[i]].replace(' ', ''))
print("Top 3邻居：", result2)

32 0.25 支持斯科拉里
17 0.5078125 斯科拉里水平里皮
29 0.4921875 ​中国队斯科拉里开足薪水足需要条件
Top 3邻居： [32, 17, 29]


In [134]:
# tfidf_vectorizer = TfidfVectorizer(min_df=3, max_features=None, ngram_range=(1, 2), use_idf=1, smooth_idf=1,sublinear_tf=1)
# x_train = tfidf_vectorizer.fit_transform(documents).toarray()
# x_train[:10]

In [ ]:
# # 使用sklearn中的lshforest
# from sklearn.neighbors import LSHForest

# lshf = LSHForest(random_state = 2)
# lshf.fit(x_train)
# #获取top-k个邻居
# distances, indices = lshf.kneighbors(query2, n_neighbors=3)
# print("距离:",distances)
# print("index:",indices)